In [2]:
import pandas as pd
import tarfile
import zipfile
import gzip
import io
import csv
from data.preprocessing import interval_time, CELIAS_Proton_Monitor_data

In [3]:
scrap_date = interval_time('20200901', '20221010')
monitor = CELIAS_Proton_Monitor_data(scrap_date)

100%|██████████| 37024528/37024528 [00:15<00:00, 2363112.91it/s]


KeyError: "There is no item named '{year}_CELIAS_Proton_Monitor_5min.txt' in the archive"

In [ ]:
monitor

ftp://sohoftp.nascom.nasa.gov/qkl/lasco/quicklook/level_05/
